### Run the cell below and update the Mongo Host and Port Info

In [1]:
! bash ~/dev/PySyft/scripts/dev_tools.sh

-----------------------------
Containers | Exposed Ports
-----------------------------
/lucid_hutter-seaweedfs-1  57447 57448 
/lucid_hutter-redis-1  57443 
/lucid_hutter-queue-1  57442 
/lucid_hutter-mongo-1  57440 
/lucid_hutter-jaeger-1  14268 57441 


In [2]:
## Start here.

In [3]:
# Set your public ports for mongo here.

class EnvSettings:
    MONGO_HOST = "localhost"  # # change to 'localhost'
    MONGO_PORT = "57440"  # change to public port
    MONGO_USERNAME = "root"
    MONGO_PASSWORD = "example"  # nosec
    
env_settings = EnvSettings()

In [4]:
import syft as sy
from syft.core.node.new.user import UserCreate, User, UserUpdate
from syft.core.node.new.mongo_document_store import MongoBsonObject
from syft.core.node.new.mongo_document_store import MongoDocumentStore, MongoStoreConfig
from syft.core.node.new.document_store import StoreClientConfig
from syft.core.common.uid import UID
from syft.core.node.new.user_stash import UserStash

In [5]:
mongo_client_config = StoreClientConfig(
    hostname=env_settings.MONGO_HOST,
    port=env_settings.MONGO_PORT,
    username=env_settings.MONGO_USERNAME,
    password=env_settings.MONGO_PASSWORD,
)

In [6]:
store = MongoDocumentStore(client_config=mongo_client_config)

In [7]:
from syft.core.node.new.dict_document_store import DictDocumentStore

In [8]:
dict_store = DictDocumentStore()

In [9]:
mongo_store_ser = sy.serialize(store, to_bytes=True)

In [10]:
store = sy.deserialize(mongo_store_ser, from_bytes=True)

In [11]:
user_stash = UserStash(store=store)

In [12]:
store_partition = store.partition(settings=user_stash.settings)

In [13]:
store_partition

In [14]:
desered_store_partition = sy.deserialize(sy.serialize(store_partition, to_bytes=True), from_bytes=True)

In [15]:
user_stash = sy.deserialize(sy.serialize(user_stash, to_bytes=True), from_bytes=True)

In [16]:
user_stash.partition.collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'User_index_name': {'v': 2,
  'key': [('email', 1), ('signing_key', 1), ('verify_key', 1)],
  'unique': True}}

In [17]:
new_user = UserCreate(
    name="Shubham", email="alice1@openmined.org", password="pass", password_verify="pass"
)

In [18]:
user = new_user.to(User)

In [19]:
bson_dict = user.to(MongoBsonObject)

In [20]:
bson_dict.to(UserCreate)

```python
class User:
  id: str = b42441386f7e47ceba621e4c4b74e340
  email: str = "alice1@openmined.org"
  name: str = "Shubham"
  hashed_password: str = "$2b$12$.4dpnDDW41MaIgurnewhuu8lV99h0uhXtzEZX3.gXybDWDhOVNpDS"
  salt: str = "$2b$12$.4dpnDDW41MaIgurnewhuu"
  signing_key: str = 620e1c457cf9a02e80c898b4c6b548e10211ae7f5cd73bc0457e69211b7e1d51
  verify_key: str = 0185d96d8569e184cc9f7623718ec3dd83ed3651d5d01cca28e4bcca74234dfe
  role: str = ServiceRole.GUEST
  institution: str = None
  website: str = None
  created_at: str = None

```

In [21]:
response = user_stash.set(user)

In [22]:
response

Ok(syft.core.node.new.user.User)

In [23]:
# #### Create indexes
# from pymongo import IndexModel, ASCENDING, DESCENDING
# user_stash.partition.db_collection.create_index([
#     ("email", ASCENDING),
#     ("verify_key", ASCENDING)
# ], unique=True)

In [24]:
search_result = user_stash.get_by_email(email="alice1@openmined.org")

In [25]:
search_result.ok() if search_result.is_ok() else search_result.err()

```python
class User:
  id: str = b42441386f7e47ceba621e4c4b74e340
  email: str = "alice1@openmined.org"
  name: str = "Shubham"
  hashed_password: str = "$2b$12$.4dpnDDW41MaIgurnewhuu8lV99h0uhXtzEZX3.gXybDWDhOVNpDS"
  salt: str = "$2b$12$.4dpnDDW41MaIgurnewhuu"
  signing_key: str = 620e1c457cf9a02e80c898b4c6b548e10211ae7f5cd73bc0457e69211b7e1d51
  verify_key: str = 0185d96d8569e184cc9f7623718ec3dd83ed3651d5d01cca28e4bcca74234dfe
  role: str = ServiceRole.GUEST
  institution: str = None
  website: str = None
  created_at: str = None

```

In [26]:
uid_to_update = UID.from_string("49c3089eedda4e66adbe1142c4bea9a9")

In [27]:
update_user = UserUpdate(id=uid_to_update, email="alice1@openmined.org", name="MyBob2", institution="OpenMined")

In [28]:
update_result = user_stash.update(user=update_user.to(User))

In [29]:
update_result.ok() if update_result.is_ok() else update_result.err()

"No object found with query key: key='id' type_=<class 'syft.core.common.uid.UID'> value=<UID: 49c3089eedda4e66adbe1142c4bea9a9>"

In [30]:
user_stash.get_by_email(email="bob@alice.com")

Ok(None)

In [31]:
uid_to_delete = UID.from_string("49c3089eedda4e66adbe1142c4bea9a9")
response = user_stash.delete_by_uid(uid=uid_to_delete)

In [32]:
response

Err("Failed to delete object with qk: key='id' type_=<class 'syft.core.common.uid.UID'> value=<UID: 49c3089eedda4e66adbe1142c4bea9a9>")

### Testing the services

In [33]:
import syft as sy
from syft.core.node.new.user import UserSearch, UserCreate
from syft.core.node.worker import Worker
from syft.core.node.new.document_store import StoreClientConfig
from syft.core.common.uid import UID

In [34]:
# Set your public ports for mongo here.

class EnvSettings:
    MONGO_HOST = "localhost"  # # change to 'localhost'
    MONGO_PORT = "57440"  # change to public port
    MONGO_USERNAME = "root"
    MONGO_PASSWORD = "example"  # nosec
    
env_settings = EnvSettings()
mongo_client_config = StoreClientConfig(
    hostname=env_settings.MONGO_HOST,
    port=env_settings.MONGO_PORT,
    username=env_settings.MONGO_USERNAME,
    password=env_settings.MONGO_PASSWORD,
)

In [35]:
worker = Worker(store_config=MongoStoreConfig(client_config=mongo_client_config))

Starting Worker: Stupefied Goertzel - 643ecdd460b44b0da4b1a893ae31636d [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>]


In [36]:
worker_ser = sy.serialize(worker, to_bytes=True)

In [37]:
worker_deser = sy.deserialize(worker_ser, from_bytes=True)

In [38]:
client = sy.new_login(node=worker_deser, email="info@openmined.org", password="changethis", port=8081)

Logged into Stupefied Goertzel as <info@openmined.org>


In [39]:
u_create = UserCreate(name="Shubham", email="shubh@openmined.org", password="pass", password_verify="pass")

In [40]:
client.api.services.user.create(user_create=u_create)

<class 'syft.core.node.new.response.SyftError'>: User already exists with email: shubh@openmined.org

In [41]:
client.api.services.user.search(name="Shubham")

,type,id,name,email
0,syft.core.node.new.user.UserView,2997a13a2cad40edb298d8661301455b,Shubham,alice1@openmined.org
1,syft.core.node.new.user.UserView,b42441386f7e47ceba621e4c4b74e340,Shubham,alice1@openmined.org


In [42]:
client.api.services.user.search(name="Shubham")

,type,id,name,email
0,syft.core.node.new.user.UserView,2997a13a2cad40edb298d8661301455b,Shubham,alice1@openmined.org
1,syft.core.node.new.user.UserView,b42441386f7e47ceba621e4c4b74e340,Shubham,alice1@openmined.org


In [48]:
uid_to_delete = UID("2997a13a2cad40edb298d8661301455b")
client.api.services.user.delete(uid_to_delete)

<class 'syft.core.node.new.response.SyftSuccess'>: ID: 2997a13a2cad40edb298d8661301455b deleted

In [44]:
x = client.api.services.user.search(name="Shubham")

In [45]:
x

,type,id,name,email
0,syft.core.node.new.user.UserView,2997a13a2cad40edb298d8661301455b,Shubham,alice1@openmined.org
1,syft.core.node.new.user.UserView,b42441386f7e47ceba621e4c4b74e340,Shubham,alice1@openmined.org


In [50]:
client.api.services.user.search(name="Shubham")[0]

```python
class UserView:
  id: str = b42441386f7e47ceba621e4c4b74e340
  email: str = "alice1@openmined.org"
  name: str = "Shubham"
  role: str = ServiceRole.GUEST
  password: str = None
  password_verify: str = None
  verify_key: str = None
  institution: str = None
  website: str = None

```

In [47]:
worker2 = Worker()

Starting Worker: Beautiful Littman - 698d9d3fcabb4b75b9b982c087f8cfeb [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>]
